In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

raw_data = pd.read_excel("../data/adjusted-labels-multiclass.xlsx")

raw_data.dropna(subset=['Sentence'], inplace=True)  # Get rid of anything NaN

sentences = raw_data["Sentence"]
labels = raw_data.drop(columns=["Sentence"])

In [2]:
vectorizer = TfidfVectorizer()

sentences_tfidf = vectorizer.fit_transform(sentences)

In [3]:
svm_classifier = SVC(kernel="linear")
multi_label_svm = MultiOutputClassifier(svm_classifier)

In [4]:
from sklearn.model_selection import KFold

In [5]:
n_folds = 10
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

train_sentences, test_sentences, train_labels, test_labels = train_test_split(sentences_tfidf, labels, test_size=0.2, random_state=47)

accuracy_scores = cross_val_score(multi_label_svm, train_sentences, train_labels, cv=kf, scoring='accuracy')

print("Using ", n_folds, " Folds")
print("Mean Accuracy across the folds during training: ", accuracy_scores.mean())
# Step 6: Test the Model
y_pred = multi_label_svm.fit(train_sentences, train_labels).predict(test_sentences)

# Calculate metrics
accuracy = accuracy_score(test_labels, y_pred)
micro_precision = precision_score(test_labels, y_pred, average='micro')
macro_precision = precision_score(test_labels, y_pred, average='macro')
micro_recall = recall_score(test_labels, y_pred, average='micro')
macro_recall = recall_score(test_labels, y_pred, average='macro')

print("Micro Precision:", micro_precision)
print("Macro Precision:", macro_precision)
print("Micro Recall:", micro_recall)
print("Macro Recall:", macro_recall)

# Classification Report
print("\nClassification Report:")
print("Overall Accuracy:", accuracy)
print(classification_report(test_labels, y_pred, zero_division=0))

Using  10  Folds
Mean Accuracy across the folds during training:  0.7538759689922482
Micro Precision: 0.9624505928853755
Macro Precision: 0.9853897217857966
Micro Recall: 0.8935779816513761
Macro Recall: 0.8060973843006827

Classification Report:
Overall Accuracy: 0.7792207792207793
              precision    recall  f1-score   support

           0       0.92      0.99      0.96       440
           1       0.92      0.67      0.77        18
           2       1.00      0.71      0.83        21
           3       1.00      0.66      0.80        62
           4       1.00      0.83      0.90        69
           5       1.00      0.76      0.87        38
           6       0.99      0.92      0.96       143
           7       1.00      0.86      0.93       146
           8       1.00      0.96      0.98        56
           9       1.00      0.72      0.84        57
          10       1.00      0.78      0.87        40

   micro avg       0.96      0.89      0.93      1090
   macro avg